# Imports

In [ ]:
from pathlib import Path
from tqdm import tqdm_notebook as tqdm
import numpy as np, pandas as pd, matplotlib.pyplot as plt, plotly.express as px

## Paths

In [ ]:
rmsd_path = Path("/Users/alexpayne/lilac-mount-point/asap-datasets/retro_docking/sars_fragalysis_retrospective/20221208_analysis/")

## Import Raw Data

In [ ]:
rmsd_arrays = [np.load(npy_file) for npy_file in tqdm(list(rmsd_path.glob("*.npy")))]

# Covert arrays to dataframe and save as single npy file!

In [ ]:
arrays = np.concatenate(rmsd_arrays)

In [ ]:
np.shape(arrays)

In [ ]:
arrays[-1]

In [ ]:
cmpd_ids = [npy_path.stem for npy_path in list(rmsd_path.glob("*.npy"))]

In [ ]:
cmpd_ids

In [ ]:
test_column = np.array([[cmpd_ids[0]]*10]).T

In [ ]:
np.hstack((test_column, test_column))

In [ ]:
appended_arrays = [np.hstack((np.array([[cmpd_ids[i]]*len(array)]).T, array)) for i, array in enumerate(rmsd_arrays)]

In [ ]:
appended_arrays[0]

## i just realized I don't actually need to do this

## Converting to dataframe

In [ ]:
df = pd.DataFrame(arrays)

In [ ]:
cmpd_ids = df[0].apply(lambda x:x.split("_Mpro")[0])

In [ ]:
datasets = df[0].apply(lambda x:"_".join(x.split("_")[1:]))

In [ ]:
import yaml

In [ ]:
with open("../../../../metadata/cmpd_to_frag.yaml") as f:
    cmpd_to_frag = yaml.safe_load(f)

In [ ]:
cmpd_to_frag

In [ ]:
frag_to_cmpd = {frag:cmpd for cmpd, frag in cmpd_to_frag.items()}

In [ ]:
ref_ligands = datasets.apply(lambda x: frag_to_cmpd[x.split("_")[0]])

In [ ]:
ref_ligands

In [ ]:
pretty_df = pd.DataFrame({"Compound_ID": cmpd_ids, "Dataset": datasets, "Reference_Ligands":ref_ligands, "RMSD": df[1]})

In [ ]:
pretty_df.sort_values("Compound_ID").reset_index(drop=True)

In [ ]:
sum(pretty_df["Compound_ID"] == pretty_df["Reference_Ligands"])

In [ ]:
pretty_df.to_csv(rmsd_path / "combined_array.csv")

# Load in df

In [ ]:
df = pd.read_csv(rmsd_path / "combined_array.csv", index_col=0)

In [ ]:
df

# Add IC50s

In [ ]:
cdd_results = rmsd_path / "cdd_noncovalent_02_10_23_unfiltered.csv"

In [ ]:
cdd_results.exists()

## load in csv file

In [ ]:
cdd = pd.read_csv(cdd_results)

## function to filter cdd courtesy of ChatGPT

In [ ]:
import pandas as pd

def filter_dataframe_by_column_values(df, column_name, values_list):
    """
    Filter a pandas DataFrame by values in a certain column that are found in a list.

    Args:
        df: Pandas DataFrame to filter
        column_name: Name of the column to filter on
        values_list: List of values to filter on

    Returns:
        Pandas DataFrame containing all rows where the specified column 
        contains one of the values in the list, with only the first occurrence of each value kept.
    """
    filtered_df = df[df[column_name].isin(values_list)]
    return filtered_df.drop_duplicates(subset=column_name, keep='first')

In [ ]:
filtered_cdd = filter_dataframe_by_column_values(cdd, "Canonical PostEra ID", df.Compound_ID.unique())

In [ ]:
filtered_cdd = filtered_cdd.rename(columns={'Canonical PostEra ID': 'Compound_ID'})

In [ ]:
filtered_cdd

In [ ]:
pic50s = filtered_cdd['ProteaseAssay_Fluorescence_Dose-Response_Weizmann: Avg pIC50']

In [ ]:
pic50s_numbers = []
pic50s_types = []
for pic50 in pic50s:
    list_ = str(pic50).split(" ")
    if len(list_) == 1:
        if list_[0] == 'nan':
            pic50_float = np.nan
            pic50_type = "NA"
        else:
            pic50_float = float(list_[0])
            pic50_type = "within"
    elif len(list_) == 2:
        if list_[0] == ">":
            pic50_type = "above"
        elif list_[0] == "<":
            pic50_type = "above"
        else:
            raise NotImplementedError
        pic50_float = float(list_[1])
    else:
        raise NotImplementedError
    pic50s_numbers.append(pic50_float)
    pic50s_types.append(pic50_type)

In [ ]:
cdd_df = pd.DataFrame({"Compound_ID": filtered_cdd["Compound_ID"],
                       "pIC50": np.array(pic50s_numbers).astype(float),
                       "Assay_Range": pic50s_types}
                       )

In [ ]:
cdd_df

## add cdd results to RMSDs

In [ ]:
merged_df = pd.merge(df, 
                     cdd_df, 
                     on=["Compound_ID"], 
                     how='outer')

In [ ]:
merged_df

In [ ]:
merged_df = merge_dataframes_on_key(df, filtered_cdd, ["Compound_ID"], ["ProteaseAssay_Fluorescence_Dose-Response_Weizmann: Avg pIC50"])

In [ ]:
merged_df

In [ ]:
filtered_cdd[["Canonical PostEra ID", "ProteaseAssay_Fluorescence_Dose-Response_Weizmann: Avg pIC50"]]

In [ ]:
merged_df.sort_values(["Dataset", "RMSD"])

In [ ]:
self_docked = merged_df[merged_df["Compound_ID"] == merged_df["Reference_Ligands"]]

In [ ]:
self_docked_trimmed = self_docked[["Reference_Ligands", "pIC50", "Assay_Range"]]

In [ ]:
self_docked_trimmed.columns = ["Reference_Ligands", "Reference_pIC50", "Reference_Assay_Range"]

In [ ]:
self_docked_trimmed

In [ ]:
merged_df

In [ ]:
self_docked_trimmed[self_docked_trimmed['Reference_Ligands'] =="EDJ-MED-76744c27-4"]

In [ ]:
merged_df_with_reference = pd.merge(merged_df, self_docked_trimmed, on=["Reference_Ligands"])

In [ ]:
ranked_df = merged_df_with_reference.copy()

In [ ]:
ranked_df["RMSD_Rank"] = merged_df_with_reference.sort_values(
    ["Reference_Ligands", "RMSD"]).groupby(
    'Reference_Ligands')['RMSD'].rank('dense')

In [ ]:
ranked_df["pIC50_Rank"] = merged_df_with_reference.sort_values(
    ["Reference_Ligands", "pIC50"]).groupby(
    'Reference_Ligands')['pIC50'].rank('dense')

In [ ]:
ranked_df.sort_values(["Reference_Ligands", "RMSD_Rank"]).groupby(["Reference_Ligands"]).head(10)

## save the ranked dataframe

In [ ]:
ranked_df.to_csv(rmsd_path / "ranked_dataframe.csv")

# Play with some visualization

## load ranked_df

In [ ]:
df = pd.read_csv(rmsd_path / "ranked_dataframe.csv")

In [ ]:
test = df[df["Compound_ID"] == "ADA-UCB-6c2cb422-1"]
fig = px.scatter(test, x="RMSD_Rank", y="Reference_pIC50")

In [ ]:
fig.show()

In [ ]:
fig = px.density_heatmap(df, x="RMSD_Rank", y="pIC50_Rank")

In [ ]:
fig = px.density_heatmap(df.sample(n=40000), x="RMSD", y="pIC50_Rank", hover_data=["Compound_ID", "Reference_Ligands"])

In [ ]:
fig.show()

## Before diving into this further I want to outline what it is I actually want to learn from this